In [13]:
%matplotlib inline
from stack_nlp import *
from jupyter_mplsettings import *
matplotlib.rc('font', **font)
from glob import glob

In [14]:
cfg = local_import("./laptop_fit.py")

In [15]:
PrepareData(cfg)
data = cfg.data
data.keys()
qs = data["meta"]

Using meta path: /home/alex/data/stackexchange/overflow/caches/posts_2016.hdf5
Shape of question df (2338187, 21)
Shape of merged question df (1000000, 34)
Selecting only questions with at least 5 meaningful words.
This removes 9507 questions.
Removing bad values with missing feature information.
This affects 14 questions.
Shape of answer df (3114299, 21)
Information from answer df was merged into question df, but original df is trying to be closed and deleted from memory! Please change the config options to keep it open!
Calculating normalized columns. They are available under usual column name + _norm.


In [3]:
conn = sqlite3.connect(cfg.posts_path)

In [4]:
allmetas = sorted(glob(os.path.join(cfg.paths["metas"], "posts_*.hdf5")))
print allmetas

['/home/alex/data/stackexchange/overflow/caches/posts_2008.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2009.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2010.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2011.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2012.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2013.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2014.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2015.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2016.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2017.hdf5']


In [5]:
from gensim.parsing import preprocessing
import gensim

In [7]:
def GetPosts(n=1000, clean=True):
    i = 0
    for post in YieldDBPosts(conn):
        yield preprocessing.preprocess_string(post[0])
        i += 1
        if i == n:
            break

In [18]:
ranids = qs.Id.sample(1000)
print ranids.head()

171169    35082591
17323     34607284
756948    36895972
858547    37211298
396884    35780838
Name: Id, dtype: int64


In [35]:
posts = GetDBPosts(ranids.values, conn)
posts_w2vec = [preprocessing.preprocess_string(p) for p in posts]
# for doc2vec with document tagging
posts_d2vec = [gensim.models.doc2vec.TaggedDocument(preprocessing.preprocess_string(p), [ip]) for ip, p in enumerate(posts)]

In [36]:
posts_d2vec[0]

TaggedDocument(words=[u'specif', u'need', u'code', u'insert', u'retract', u'detector', u'microscop', u'button', u'palett', u'stem', u'detector', u'control', u'exist', u'menu', u'command', u'choosemenuitem', u'help', u'separ', u'plugin', u'specif', u'document', u'gener', u'know', u'want', u'function', u'command'], tags=[0])

In [37]:
w2vmodel = gensim.models.word2vec.Word2Vec(posts_w2vec, size=100, window=8, min_count=5, workers=8)

In [38]:
d2vmodel = gensim.models.doc2vec.Doc2Vec(posts_d2vec, size=100, window=8, min_count=5, workers=8)

In [41]:
d2vmodel.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])

array([ 0.0398253 ,  0.00171109, -0.0207583 , -0.01724119, -0.00508319,
       -0.01235079, -0.00091251,  0.00448346,  0.00613547, -0.0115783 ,
        0.01606569, -0.02848017,  0.00690961,  0.00598478, -0.01577696,
        0.01440249,  0.02188496, -0.02098961,  0.00302723, -0.00944306,
        0.00134087,  0.02193254,  0.0186031 ,  0.01686829,  0.00723156,
        0.01225144,  0.00345955,  0.0091543 , -0.02375556,  0.02526844,
       -0.00622287,  0.01853042,  0.01661909,  0.01087822,  0.01764847,
       -0.00145241,  0.00493232, -0.0145319 , -0.00364953,  0.0061184 ,
        0.00721715, -0.01718108,  0.01971325, -0.00148774, -0.0016113 ,
       -0.02190721, -0.01788802,  0.01885497, -0.01162282,  0.00071606,
       -0.00735294, -0.03986878,  0.02473359, -0.01594229,  0.01948441,
       -0.01198993,  0.01832322, -0.00342439,  0.01572293, -0.01730493,
       -0.01729368,  0.01751504,  0.0029443 , -0.02560235, -0.00137853,
        0.01380401,  0.0042604 ,  0.01323836, -0.01871311,  0.01

In [31]:
w2vmodel.wv.vocab

{u'buff': <gensim.models.keyedvectors.Vocab at 0x7f90258f5c90>,
 u'represent': <gensim.models.keyedvectors.Vocab at 0x7f9025933a10>,
 u'saver': <gensim.models.keyedvectors.Vocab at 0x7f902597c4d0>,
 u'prefix': <gensim.models.keyedvectors.Vocab at 0x7f905cba8490>,
 u'nsdate': <gensim.models.keyedvectors.Vocab at 0x7f90258d4250>,
 u'sleep': <gensim.models.keyedvectors.Vocab at 0x7f90258d4290>,
 u'glibcxx': <gensim.models.keyedvectors.Vocab at 0x7f902597c510>,
 u'typeerror': <gensim.models.keyedvectors.Vocab at 0x7f902598c5d0>,
 u'const': <gensim.models.keyedvectors.Vocab at 0x7f902597c550>,
 u'swal': <gensim.models.keyedvectors.Vocab at 0x7f90259128d0>,
 u'sorri': <gensim.models.keyedvectors.Vocab at 0x7f9025912910>,
 u'swap': <gensim.models.keyedvectors.Vocab at 0x7f902598cb50>,
 u'preprocess': <gensim.models.keyedvectors.Vocab at 0x7f9025912950>,
 u'previouscontentview': <gensim.models.keyedvectors.Vocab at 0x7f902597c5d0>,
 u'spec': <gensim.models.keyedvectors.Vocab at 0x7f902597c610>